In [1]:
!pip install sacrebleu transformers torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 2.2 MB/s eta 0:00:00


In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import sacrebleu
import torch
import json

# Load the new evaluation dataset
new_dataset_path = '/content/New_Evaluation_Dataset.json'
with open(new_dataset_path, 'r') as file:
    dataset = json.load(file)

# Separate the dataset into inputs and references
inputs = [item['text'] for item in dataset]
references = [item['code'] for item in dataset]

# Load your Hugging Face model and tokenizer
model_name = "AshArya/ITRLTrained"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, from_tf = True)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# List to store predictions
predictions = []

# Generate predictions for each input in the dataset
for input_text in inputs:
    # Tokenize the input data
    inputs_tokenized = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

    # Generate predictions
    with torch.no_grad():
        output = model.generate(**inputs_tokenized)

    # Decode the prediction into text and store it in the predictions list
    prediction = tokenizer.decode(output[0], skip_special_tokens=True)
    predictions.append(prediction)

# Calculate the BLEU score for the entire dataset
bleu = sacrebleu.corpus_bleu(predictions, [references])
print(f"BLEU score: {bleu.score}")

# Calculate the Exact Match score for the entire dataset
exact_matches = sum([1 if pred.strip() == ref.strip() else 0 for pred, ref in zip(predictions, references)])
exact_match_score = exact_matches / len(references) * 100
print(f"Exact Match score: {exact_match_score:.2f}%")


tf_model.h5:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

All TF 2.0 model weights were used when initializing T5ForConditionalGeneration.

Some weights of T5ForConditionalGeneration were not initialized from the TF 2.0 model and are newly initialized: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/21.0k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/639k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/294k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/15.5k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


BLEU score: 87.64200982025153
Exact Match score: 81.62%
